In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input 
import numpy as np
from PIL import Image
import os

directories = ['feature_vectors_iv3','feature_vectors_iv3/train','feature_vectors_iv3/test']

for dir in directories:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [ ]:
class FeatureExtractor:
    def __init__(self):
        
        self.model = InceptionV3(weights='imagenet',include_top=False,pooling='max')
        

    def extract(self, img):
        img = img.resize((299, 299))  
        img = img.convert('RGB')
        
        img1 = img.copy()
        data = img1.getdata()

        r = [(d[0], 0, 0) for d in data]
        g = [(0, d[1], 0) for d in data]
        b = [(0, 0, d[2]) for d in data]

        img1.putdata(r)
        display(img1)
        
        img1.putdata(g)
        display(img1)
        
        img1.putdata(b)
        display(img1)
        
        x = image.img_to_array(img)  
        x = np.expand_dims(x, axis=0)  
        x = preprocess_input(x)  
        feature = self.model.predict(x)[0]  
        return feature / np.linalg.norm(feature) 

In [ ]:
from pathlib import Path
fe = FeatureExtractor()

i = 0
for img_path in sorted(Path("./images_aggregate").glob("*.jpg")):
    i+=1
    feature = fe.extract(img=Image.open(img_path))
    
    if(i%5==0 and i%10!=0):
        feature_path = Path("./feature_vectors_iv3/test") / (img_path.stem + ".npy")  
        np.save(feature_path, feature)
    else:
        feature_path = Path("./feature_vectors_iv3/train") / (img_path.stem + ".npy")  
        np.save(feature_path, feature)